In [1]:
import pandas as pd # begin if read data from three files
import numpy as np

# read 3 part features of predicting data by 3 different types to save memory
X=pd.read_csv('./Xint8_predictlevel123.dat', sep = '\t', dtype=np.int8)
Xint16=pd.read_csv('./Xint16_predictlevel123.dat', sep = '\t', dtype=np.int16)
X_gender=pd.read_csv('./X_gender_predictlevel123.dat', sep = '\t')

# read header of int16 type features
with open('headers123.small.txt') as f:
    list16 = f.read().splitlines()

# add features from another two files to X
X[list16]=Xint16[list16]
X['gender']=X_gender['gender']
X

# read the Patient_ID column to write into result file (used only when read X from 3 files)
df_training_data = pd.read_csv('./trn_final_predict_with15.txt', sep = '\t')
y_patient_col = df_training_data[['Patient_ID','Buy_Diabetes']]

,2011A,2011A01,2011A01A,2011A02,2011A02A,2011A02B,2011A03,2011A03A,2011A03B,2011A03F,...,year_of_birth,postcode,trans_all,trans_lipids,trans_hypertension,trans_Depression,trans_Epilepsy,trans_COPD,trans_Diabetes,gender
0,11,0,0,11,3,8,0,0,0,0,...,1900,3939,73,30,30,0,0,0,0,M
1,5,0,0,5,0,5,0,0,0,0,...,1900,3146,56,27,0,0,0,0,0,F
2,0,0,0,0,0,0,0,0,0,0,...,1900,2470,54,37,0,0,0,0,0,M
3,6,0,0,6,0,6,0,0,0,0,...,1900,3109,0,0,0,0,0,0,0,M
4,0,0,0,0,0,0,0,0,0,0,...,1900,3677,9,0,6,0,3,0,0,M
5,0,0,0,0,0,0,0,0,0,0,...,1900,3996,0,0,0,0,0,0,0,F
6,7,0,0,0,0,0,0,0,0,0,...,1900,2452,120,54,0,32,0,0,34,M
7,2,0,0,0,0,0,0,0,0,0,...,1900,2264,23,0,0,0,0,0,23,F
8,0,0,0,0,0,0,0,0,0,0,...,1900,3810,0,0,0,0,0,0,0,F
9,0,0,0,0,0,0,0,0,0,0,...,1900,2650,40,0,0,16,9,0,0,F


In [2]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279152 entries, 0 to 279151
Columns: 2185 entries, 2011A to gender
dtypes: int16(39), int8(2145), object(1)
memory usage: 593.9+ MB


In [3]:
#change X to dictionary and vectorize it

import sys
import os
import pickle
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.cross_validation import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from itertools import tee, islice, chain
%matplotlib inline


X = X.T.to_dict().values()


# Verctirize X to vectors
# read vectorizer create by trainer to make sure has same number of features
# here use transform() instead of fix_transform() which used in trainer
with open('./vectorizer.dat', 'rb') as vectorizer_f:
    vectorizer = pickle.load(vectorizer_f)
X = vectorizer.transform(X)
print(X)

# load the saved classifier
classifier_path = './classifier.dat'
with open(classifier_path, 'rb') as classifier_f:
    classifier = pickle.load(classifier_f)
print('classifier: ',classifier)

In [4]:
#  predict result       
#predict_y = classifier.predict(X.toarray()) # get 0/1
# get probability
predict_y = classifier.predict_proba(X.toarray())
predict_y = predict_y[:,1]

print(predict_y)
print(type(predict_y))

[ 0.01736605  0.01472732  0.01839862 ...,  0.01562578  0.01115305
  0.76584176]
<class 'numpy.ndarray'>


In [5]:
# get the format final result file need
y_patient_col[['Patient_ID','Buy_Diabetes']] = df_training_data[['Patient_ID','Buy_Diabetes']]
print(type(y_patient_col))
print(predict_y)
y_patient_col['Diabetes'] = predict_y.tolist()
print(y_patient_col)
y_patient_col = y_patient_col[['Patient_ID','Diabetes']]
y_patient_col


# write to file
path_to_results = './prob_dbgbmatc123_cout.csv'
y_patient_col.to_csv(path_to_results,index=False)

<class 'pandas.core.frame.DataFrame'>
[ 0.01736605  0.01472732  0.01839862 ...,  0.01562578  0.01115305
  0.76584176]
        Patient_ID  Buy_Diabetes  Diabetes
0           279201             0  0.017366
1           279202             0  0.014727
2           279203             0  0.018399
3           279204             0  0.020009
4           279205             0  0.015694
5           279206             0  0.012598
6           279207             1  0.939570
7           279208             1  0.758512
8           279209             0  0.014335
9           279210             0  0.012578
10          279211             0  0.014682
11          279212             0  0.016724
12          279213             0  0.022295
13          279214             0  0.010691
14          279215             0  0.016012
15          279216             0  0.018291
16          279217             0  0.014862
17          279218             0  0.019292
18          279219             0  0.010273
19          279220    

,Patient_ID,Diabetes
0,279201,0.017366
1,279202,0.014727
2,279203,0.018399
3,279204,0.020009
4,279205,0.015694
5,279206,0.012598
6,279207,0.939570
7,279208,0.758512
8,279209,0.014335
9,279210,0.012578


In [4]:
# read data

import pandas as pd
import os
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.plotly as py
import numpy as np

# datas have been drop duplicate and collected in SQL Server
# except ATClevelcode part, since pandas pivot_table is a better tool to deal with it

# read features of training dataset except features about ATClevelcode
df_training_data = pd.read_csv('./trn_final_predict_with15.txt', sep = '\t') #change

# read ATClevelcode(level 1-3) associated data from training dataset
# only (Patient_ID,year_atclevel,cnt) columns, need to be aggregated in next step
df_atc_data = pd.read_csv('./trn_predicting_atc123.txt', sep = '\t') #change

# read the Patient_ID column to write into result file  
y_patient_col = df_training_data[['Patient_ID','Buy_Diabetes']]

df_atc_data

,Patient_ID,year_atclevel,cnt
0,0,2012L01D,0
1,0,2013L01C,0
2,0,2014A16,0
3,0,2014A16A,0
4,0,Q2V07,0
5,0,Q2V07A,0
6,0,2012S01H,0
7,0,2014A14,0
8,0,Q1A07B,0
9,0,Q1A16A,0


In [5]:
df_training_data

,Patient_ID,gender,year_of_birth,postcode,state_code,trans_all,trans_lipids,trans_hypertension,trans_Depression,trans_Immunology,...,dur_Heart_Failure_15,trans_Epilepsy_15,max_time_Epilepsy_15,dur_Epilepsy_15,trans_COPD_15,max_time_COPD_15,dur_COPD_15,trans_Diabetes_15,max_time_Diabetes_15,dur_Diabetes_15
0,279201,M,1900,3939,3,73,30,30,0,0,...,1,0,2015-01-01,0,0,2015-01-01,0,0,2015-01-01,0
1,279202,F,1900,3146,3,56,27,0,0,0,...,211,0,2015-01-01,0,0,2015-01-01,0,0,2015-01-01,0
2,279203,M,1900,2470,2,54,37,0,0,0,...,155,0,2015-01-01,0,0,2015-01-01,0,0,2015-01-01,0
3,279204,M,1900,3109,3,0,0,0,0,0,...,0,0,2015-01-01,0,0,2015-01-01,0,0,2015-01-01,0
4,279205,M,1900,3677,3,9,0,6,0,0,...,0,0,2015-01-01,0,0,2015-01-01,0,0,2015-01-01,0
5,279206,F,1900,3996,3,0,0,0,0,0,...,0,0,2015-01-01,0,0,2015-01-01,0,0,2015-01-01,0
6,279207,M,1900,2452,2,120,54,0,32,0,...,0,0,2015-01-01,0,0,2015-01-01,0,6,2015-09-20,253
7,279208,F,1900,2264,2,23,0,0,0,0,...,0,0,2015-01-01,0,0,2015-01-01,0,3,2015-05-17,134
8,279209,F,1900,3810,3,0,0,0,0,0,...,0,0,2015-01-01,0,0,2015-01-01,0,0,2015-01-01,0
9,279210,F,1900,2650,2,40,0,0,16,0,...,302,0,2015-01-01,0,0,2015-01-01,0,0,2015-01-01,0


In [6]:
# pivote table, the values of year_atclevel as column name
# year_atlevel including the combination of (2011-2014,2015Q1-2015Q4) and (all ATC level 1-3 codes)
pv_table = pd.pivot_table(df_atc_data,values='cnt',index=['Patient_ID'],
                          columns=['year_atclevel'],aggfunc='sum').fillna(0).astype(int)
X=pd.DataFrame(pv_table.to_records())
X

,Patient_ID,2011A,2011A01,2011A01A,2011A02,2011A02A,2011A02B,2011A03,2011A03A,2011A03B,...,Q4V,Q4V03,Q4V03A,Q4V04,Q4V04C,Q4V06,Q4V06D,Q4V07,Q4V07A,inserted
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,279201,11,0,0,11,3,8,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,279202,5,0,0,5,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,279203,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,279204,6,0,0,6,0,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,279205,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,279206,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,279207,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,279208,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,279209,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# In order to make predicting data matrix has the same columns as training data
# add some records using Patient_ID=0, delete this virtual patient after pivote table
X = X[X['Patient_ID']!=0]
# some columns in predicting data matrix has values need int16, but only use int8 in training data
# set the values as max of int8 (only 5 columns is ok in more than two thousand columns)
X.loc[X['2011N02C']>=255,'2011N02C']=255
X.loc[X['2012N02C']>=255,'2012N02C']=255
X.loc[X['2012N05']>=255,'2012N05']=255
X.loc[X['2012N05B']>=255,'2012N05B']=255
X.loc[X['2013N05B']>=255,'2013N05B']=255

In [23]:
# make sure index begin from 0, so that the atc features index is same as other features 
X=X.reset_index()
X

,index,Patient_ID,2011A,2011A01,2011A01A,2011A02,2011A02A,2011A02B,2011A03,2011A03A,...,Q4V,Q4V03,Q4V03A,Q4V04,Q4V04C,Q4V06,Q4V06D,Q4V07,Q4V07A,inserted
0,1,279201,11,0,0,11,3,8,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,279202,5,0,0,5,0,5,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,279203,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,279204,6,0,0,6,0,6,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,279205,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6,279206,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,279207,7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,8,279208,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,9,279209,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,279210,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
del X['index']
X

,Patient_ID,2011A,2011A01,2011A01A,2011A02,2011A02A,2011A02B,2011A03,2011A03A,2011A03B,...,Q4V,Q4V03,Q4V03A,Q4V04,Q4V04C,Q4V06,Q4V06D,Q4V07,Q4V07A,inserted
0,279201,11,0,0,11,3,8,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,279202,5,0,0,5,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,279203,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,279204,6,0,0,6,0,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,279205,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,279206,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,279207,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,279208,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,279209,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,279210,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# add features together
X[['gender','year_of_birth','postcode','state_code','trans_all','trans_lipids','trans_hypertension','trans_Depression','trans_Immunology','trans_Urology','trans_Anti_Coagulant','trans_Osteoporosis','trans_Heart_Failure','trans_Epilepsy','trans_COPD','trans_Diabetes','Buy_Diabetes','dur_Diabetes_15']]=df_training_data[['gender','year_of_birth','postcode','state_code','trans_all','trans_lipids','trans_hypertension','trans_Depression','trans_Immunology','trans_Urology','trans_Anti_Coagulant','trans_Osteoporosis','trans_Heart_Failure','trans_Epilepsy','trans_COPD','trans_Diabetes','Buy_Diabetes','dur_Diabetes_15']]
X.loc[X['dur_Diabetes_15']>0, 'dur_Diabetes_15']=1
X

,Patient_ID,2011A,2011A01,2011A01A,2011A02,2011A02A,2011A02B,2011A03,2011A03A,2011A03B,...,trans_Immunology,trans_Urology,trans_Anti_Coagulant,trans_Osteoporosis,trans_Heart_Failure,trans_Epilepsy,trans_COPD,trans_Diabetes,Buy_Diabetes,dur_Diabetes_15
0,279201,11,0,0,11,3,8,0,0,0,...,0,0,0,0,13,0,0,0,0,0
1,279202,5,0,0,5,0,5,0,0,0,...,0,0,12,0,17,0,0,0,0,0
2,279203,0,0,0,0,0,0,0,0,0,...,0,0,0,0,17,0,0,0,0,0
3,279204,6,0,0,6,0,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,279205,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,0,0,0,0
5,279206,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,279207,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,34,1,1
7,279208,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,23,1,1
8,279209,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,279210,0,0,0,0,0,0,0,0,0,...,0,0,0,0,15,9,0,0,0,0


In [28]:
with open('headers123.small.txt') as f: # headers123.small.txt has all the feature names of int16
    list16 = f.read().splitlines()
Xint16=X[list16]
Xint16.to_csv('./Xint16_predictlevel123.dat', sep='\t', index=False)

with open('headers123.txt') as f:
    list8 = f.read().splitlines()
Xint8=X[list8]
Xint8.to_csv('./Xint8_predictlevel123.dat', sep='\t', index=False)

X_gender=df_training_data[['Patient_ID','gender']]
X_gender.to_csv('./X_gender_predictlevel123.dat', sep='\t', index=False)

Xint16

,2011C,2011N,2011N02,2011N02A,2011N07,2011N07B,2011NA,2012C,2012N,2012N02,...,2014NA,year_of_birth,postcode,trans_all,trans_lipids,trans_hypertension,trans_Depression,trans_Epilepsy,trans_COPD,trans_Diabetes
0,27,2,2,0,0,0,0,27,3,3,...,0,1900,3939,73,30,30,0,0,0,0
1,17,0,0,0,0,0,0,15,0,0,...,0,1900,3146,56,27,0,0,0,0,0
2,10,0,0,0,0,0,0,9,0,0,...,0,1900,2470,54,37,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1900,3109,0,0,0,0,0,0,0
4,6,0,0,0,0,0,0,6,0,0,...,0,1900,3677,9,0,6,0,3,0,0
5,6,0,0,0,0,0,0,6,0,0,...,0,1900,3996,0,0,0,0,0,0,0
6,6,10,4,4,0,0,0,12,9,3,...,0,1900,2452,120,54,0,32,0,0,34
7,0,0,0,0,0,0,0,0,0,0,...,0,1900,2264,23,0,0,0,0,0,23
8,0,0,0,0,0,0,0,0,0,0,...,0,1900,3810,0,0,0,0,0,0,0
9,3,6,0,0,0,0,0,4,11,0,...,0,1900,2650,40,0,0,16,9,0,0
